# load and import lib and data

In [ ]:
!pip install ktrain

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install keras==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.16.1 requires keras>=3.0.0, but you have keras 2.15.0 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [ ]:
!pip install --upgrade ktrain

In [ ]:
!pip install transformers

In [ ]:
!pip install sacremoses

In [ ]:
import pandas as pd
from copy import copy
import re
import numpy as np

In [ ]:
!wget https://github.com/google-research/google-research/raw/master/goemotions/data/test.tsv

--2024-04-29 01:48:13--  https://github.com/google-research/google-research/raw/master/goemotions/data/test.tsv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv [following]
--2024-04-29 01:48:14--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 436706 (426K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 426.47K  --.-KB/s    in 0.03s   

2024-04-29 01:48:14 (12.0 MB/s) - ‘test.tsv’ saved [436706/43670

In [ ]:
!wget https://github.com/google-research/google-research/raw/master/goemotions/data/dev.tsv

--2024-04-29 01:48:14--  https://github.com/google-research/google-research/raw/master/goemotions/data/dev.tsv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/dev.tsv [following]
--2024-04-29 01:48:14--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439059 (429K) [text/plain]
Saving to: ‘dev.tsv’

dev.tsv             100%[===================>] 428.77K  --.-KB/s    in 0.04s   

2024-04-29 01:48:14 (11.7 MB/s) - ‘dev.tsv’ saved [439059/439059]



In [ ]:
!wget https://github.com/google-research/google-research/raw/master/goemotions/data/train.tsv

--2024-04-29 01:48:14--  https://github.com/google-research/google-research/raw/master/goemotions/data/train.tsv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/train.tsv [following]
--2024-04-29 01:48:15--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3519053 (3.4M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   3.36M  --.-KB/s    in 0.07s   

2024-04-29 01:48:15 (50.4 MB/s) - ‘train.tsv’ saved [351905

In [ ]:
!wget https://github.com/google-research/google-research/raw/master/goemotions/data/emotions.txt

--2024-04-29 01:48:15--  https://github.com/google-research/google-research/raw/master/goemotions/data/emotions.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/emotions.txt [following]
--2024-04-29 01:48:16--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/emotions.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248 [text/plain]
Saving to: ‘emotions.txt’

emotions.txt        100%[===================>]     248  --.-KB/s    in 0s      

2024-04-29 01:48:16 (9.34 MB/s) - ‘emotions.txt’ saved [24

# pre-process

In [ ]:
## add name variable to a name of a model: baseline - no aug, first_aug, ...
# name = 'baseline_labse_new'
# name = 'baseline_distilbert_new'
# name = 'distilbert_dummy_copies_5classes'
# name = 'labse_dummy_copies_5classs'
# name = 'labse_augmented7'
# name = 'labse_augmented32'
# name = 'labse_augmented32'
# name = 'distilbert_augmented32'
# name = 'labse_augmented34'
name = 'labse_augmented4'
# name='labse_oversampling_3classes'

file_path_aug = 'fin_all_generated_augmentation_deepl.tsv'

In [ ]:
def convert_to_vec(el,num=28):
  lst = [0.0 for _ in range(num)]
  for num in el:
    lst[int(num)] =1
  return lst

In [ ]:
with open('emotions.txt','r') as f:
  emotions = []
  for el in f.readlines():
    emotions.append(el.strip('\n'))
print(emotions)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
train_df = pd.read_csv('train.tsv',sep='\t', header=None)
dev_df = pd.read_csv('dev.tsv',sep='\t', header=None)
test_df = pd.read_csv('test.tsv',sep='\t', header=None)

In [ ]:
print(len(train_df))

43410


**ADD AUGMENTED DATA**

In [ ]:
## load augmented data file
augmented_data_df = pd.read_csv(file_path_aug, sep='\t', header=None)
augmented_data_df[1] = augmented_data_df[1].astype(str) ## convert labels to str

## Add augmented data to the train df
train_df = pd.concat([train_df, augmented_data_df], axis = 0).reset_index(drop=True)
# train_df = train_df.sample(frac=1).reset_index(drop=True)

In [ ]:
# ## let's test on small amount of data
# train_df = train_df.sample(100,random_state=42)
# dev_df = dev_df.sample(20,random_state=42)
# test_df = test_df.sample(20,random_state=42)

In [ ]:
train_X_df,train_labels_df = train_df[0],train_df[1]
test_X_df,test_labels_df = test_df[0],test_df[1]
dev_X_df,dev_labels_df = dev_df[0],dev_df[1]

In [ ]:
print(train_labels_df.apply(type).value_counts())

1
<class 'str'>    47543
Name: count, dtype: int64


In [ ]:
dev_labs = []
for el in dev_labels_df.values:
  n_l = []
  l = re.split(',',el)
  for e in l:
    n_l.append(np.float64(e))
  dev_labs.append(copy(convert_to_vec(n_l)))
dev_labs = np.asarray(dev_labs).astype(np.float32)

In [ ]:
train_labs = []
for el in train_labels_df.values:
  n_l = []
  l = re.split(',',el)
  for e in l:
    n_l.append(np.float64(e))
  train_labs.append(copy(convert_to_vec(n_l)))
train_labs = np.asarray(train_labs).astype(np.float32)

In [ ]:
test_labs = []
for el in test_labels_df.values:
  n_l = []
  l = re.split(',',el)
  for e in l:
    n_l.append(np.float64(e))
  test_labs.append(copy(convert_to_vec(n_l)))
test_labs = np.asarray(test_labs).astype(np.float32)

In [ ]:
len(train_X_df)

47543

In [ ]:
# print(str(train_df[49871]))

In [ ]:
# train_labs[49870]

In [ ]:
# train_X_df

In [ ]:
##count emotions
for i in range(28):
    emo_df = train_df[train_df[1] == str(i)]
    # if i == 21 or i == 23 or i == 16 or i == 19 or i == 12:
    print(f"Count for emotion {i}: {len(emo_df)}")

Count for emotion 0: 2710
Count for emotion 1: 1652
Count for emotion 2: 1025
Count for emotion 3: 1451
Count for emotion 4: 1873
Count for emotion 5: 649
Count for emotion 6: 858
Count for emotion 7: 1389
Count for emotion 8: 389
Count for emotion 9: 709
Count for emotion 10: 1402
Count for emotion 11: 498
Count for emotion 12: 2070
Count for emotion 13: 510
Count for emotion 14: 430
Count for emotion 15: 1857
Count for emotion 16: 374
Count for emotion 17: 853
Count for emotion 18: 1427
Count for emotion 19: 849
Count for emotion 20: 861
Count for emotion 21: 464
Count for emotion 22: 586
Count for emotion 23: 842
Count for emotion 24: 353
Count for emotion 25: 817
Count for emotion 26: 720
Count for emotion 27: 12823


# Train

In [ ]:
import ktrain
import transformers

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:
# MODEL_NAME = "sentence-transformers/LaBSE"
MODEL_NAME = "distilbert-base-uncased"
t = ktrain.text.Transformer(MODEL_NAME, maxlen=124, class_names=emotions)
trn = t.preprocess_train(list(train_X_df), train_labs)
val = t.preprocess_test(list(dev_X_df), dev_labs)
model = t.get_classifier()

In [ ]:
print(list(train_X_df)[1])
print(list(train_labs)[1])

Now if he does off himself, everyone will think hes having a laugh screwing with people instead of actually dead
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1.]


In [ ]:
learner =  ktrain.get_learner(model, train_data=trn, val_data=val,batch_size=16) ##change batch size prev 16

In [ ]:
# learner.lr_find(max_epochs=1) ##change epochs prev 3

In [ ]:
# learner.lr_plot(suggest=True)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
import time
start_time = time.time()

In [ ]:
learner.autofit(6.91E-05, 3, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)]) # for most lr=6.91E-05 tested also 5.61E-05



begin training using triangular learning rate policy with max lr of 6.91e-05...
Epoch 1/3
3141/3141 [==============================] - 530s 146ms/step - loss: 0.1196 - binary_accuracy: 0.9651 - val_loss: 0.0857 - val_binary_accuracy: 0.9687
Epoch 2/3
3141/3141 [==============================] - 450s 142ms/step - loss: 0.0755 - binary_accuracy: 0.9724 - val_loss: 0.0811 - val_binary_accuracy: 0.9700
Epoch 3/3
3141/3141 [==============================] - 451s 142ms/step - loss: 0.0661 - binary_accuracy: 0.9753 - val_loss: 0.0819 - val_binary_accuracy: 0.9703


In [ ]:
end_time = time.time()
training_time = end_time - start_time
print(f"Training Time: {training_time} seconds")

Training Time: 1431.5210299491882 seconds


In [ ]:
# learner.validate(val_data=val, class_names=t.get_classes())

# Predict

In [ ]:
test = t.preprocess_test(list(test_X_df),test_labs)

preprocessing test...
language: en
test sequence lengths:
	mean : 13
	95percentile : 24
	99percentile : 27


In [ ]:
start_time = time.time()

In [ ]:
prd = learner.predict(test)

5427/5427 [==============================] - 114s 19ms/step


In [ ]:
end_time = time.time()
prediction_time = end_time - start_time
print(f"Prediction Time: {prediction_time} seconds")

Prediction Time: 117.0933256149292 seconds


In [ ]:
import tensorflow as tf
import sklearn

In [ ]:
def conv_to01(L):
  for i,el in enumerate(L):
    if el >0.5:
      L[i] = 1
    else:
      L[i] = 0
  return L

In [ ]:
new_prd = [conv_to01(el) for el in prd]

In [ ]:
f1_score_value = sklearn.metrics.f1_score(test_labs, prd, average='macro')

In [ ]:
f1_score_value

0.49307946256234725

In [ ]:
m = tf.keras.metrics.CategoricalAccuracy()
m.update_state(test_labs, prd)
categorical_accuracy = m.result().numpy()

In [ ]:
categorical_accuracy

0.5338124

# Save results

In [ ]:
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = f'/content/drive/MyDrive/praca-mgr/models/labse_augmented4'

Mounted at /content/drive


In [ ]:
history = learner.history

history_dict = {k: [float(v) for v in values] for k, values in history.history.items()}

with open(f'{path}/training_history_{name}.json', 'w') as f:
    json.dump(history_dict, f)

In [ ]:
# learner.save_model('/content/drive/MyDrive/praca/model_v5.h5')
my_model_path =f'{path}/my_model_v1{name}'
learner.save_model(my_model_path)

In [ ]:
my_model_path2 = f'{path}/my_model_v1_with_preprocesor{name}'
ktrain.get_predictor(learner.model, t).save(my_model_path2)

In [ ]:
# Save predictions and true labels for the test set to CSV for later analysis
test_labels_df.to_csv(f"{path}/test_labels{name}.csv", index=False)
predictions_df = pd.DataFrame(prd)
predictions_df.to_csv(f"{path}/predictions{name}.csv", index=False, header=False)

In [ ]:
# Save test results to CSV
test_measures = pd.DataFrame({
    'f1_score_macro': [f1_score_value],
    'categorical_accuracy': [categorical_accuracy]
})
test_measures.to_csv(f'{path}/test_measures_{name}.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# After training, plot the training and validation loss

# Plot training & validation accuracy values
# plt.figure(figsize=(12, 4))
# plt.subplot(1, 2, 1)
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model Accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Val'], loc='upper left')
# plt.savefig(f'{path}/model_accuracy_{name}.png')  # Save the plot as a .png file

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig(f'{path}/model_loss_{name}.png')  # Save the plot as a .png file
plt.close()

In [ ]:
## saving augmentation details
with open(f'{path}/augmentation_file_details_{name}.txt','w') as f:
  f.write(file_path_aug)

In [ ]:
## save environment details
import platform
import sys
import subprocess

def pip_freeze():
    output = subprocess.run([sys.executable, '-m', 'pip', 'freeze'], stdout=subprocess.PIPE, text=True)
    return output.stdout.splitlines()

environment_details = {
    'platform': platform.platform(),
    'python_version': sys.version,
    'pip_freeze': pip_freeze()
}

with open(f'{path}/environment_details.json', 'w') as f:
    json.dump(environment_details, f, indent=4)